In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn.functional as F

We consider the problem of estimating the impact of assigning a different room as compared to what the customer had reserved on Booking Cancellations.

In [3]:
hotel_1 = pd.read_csv('H1.csv',parse_dates=True,index_col='ReservationStatusDate')
hotel_2 = pd.read_csv('H2.csv',parse_dates=True,index_col='ReservationStatusDate')
hotel_1.head(10)

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus
ReservationStatusDate,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,342,2015,July,27,1,0,0,2,0,...,3,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out
2015-07-01,0,737,2015,July,27,1,0,0,2,0,...,4,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out
2015-07-02,0,7,2015,July,27,1,0,1,1,0,...,0,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out
2015-07-02,0,13,2015,July,27,1,0,1,1,0,...,0,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out
2015-07-03,0,14,2015,July,27,1,0,2,2,0,...,0,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out
2015-07-03,0,14,2015,July,27,1,0,2,2,0,...,0,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out
2015-07-03,0,0,2015,July,27,1,0,2,2,0,...,0,No Deposit,NULL,NULL,0,Transient,107.0,0,0,Check-Out
2015-07-03,0,9,2015,July,27,1,0,2,2,0,...,0,No Deposit,303,NULL,0,Transient,103.0,0,1,Check-Out
2015-05-06,1,85,2015,July,27,1,0,3,2,0,...,0,No Deposit,240,NULL,0,Transient,82.0,0,1,Canceled


In [4]:
hotel_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40060 entries, 2015-07-01 to 2017-09-14
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   IsCanceled                   40060 non-null  int64  
 1   LeadTime                     40060 non-null  int64  
 2   ArrivalDateYear              40060 non-null  int64  
 3   ArrivalDateMonth             40060 non-null  object 
 4   ArrivalDateWeekNumber        40060 non-null  int64  
 5   ArrivalDateDayOfMonth        40060 non-null  int64  
 6   StaysInWeekendNights         40060 non-null  int64  
 7   StaysInWeekNights            40060 non-null  int64  
 8   Adults                       40060 non-null  int64  
 9   Children                     40060 non-null  int64  
 10  Babies                       40060 non-null  int64  
 11  Meal                         40060 non-null  object 
 12  Country                      39596 non-null  object 
 13 

This dataset contains booking information for a city hotel and a resort hotel taken from a real hotel in Portugal

## Process Null

In [6]:
print(hotel_1.isna().sum())
print(hotel_2.isna().sum())

IsCanceled                       0
LeadTime                         0
ArrivalDateYear                  0
ArrivalDateMonth                 0
ArrivalDateWeekNumber            0
ArrivalDateDayOfMonth            0
StaysInWeekendNights             0
StaysInWeekNights                0
Adults                           0
Children                         0
Babies                           0
Meal                             0
Country                        464
MarketSegment                    0
DistributionChannel              0
IsRepeatedGuest                  0
PreviousCancellations            0
PreviousBookingsNotCanceled      0
ReservedRoomType                 0
AssignedRoomType                 0
BookingChanges                   0
DepositType                      0
Agent                            0
Company                          0
DaysInWaitingList                0
CustomerType                     0
ADR                              0
RequiredCarParkingSpaces         0
TotalOfSpecialReques

In [34]:
len(hotel_1.Company.unique())

236

It can be easily seen that the null values in the Company and Agent features are not counted. Now, we need to convert them to the same way as other null value so that it will be easier to treat them later.

In [36]:
hotel_1 = hotel_1.replace(to_replace = '       NULL', 
                 value =np.NAN) 
print(hotel_1.isna().sum())
hotel_2 = hotel_2.replace(to_replace = '       NULL', 
                 value =np.NAN) 
print(hotel_2.isna().sum())

IsCanceled                         0
LeadTime                           0
ArrivalDateYear                    0
ArrivalDateMonth                   0
ArrivalDateWeekNumber              0
ArrivalDateDayOfMonth              0
StaysInWeekendNights               0
StaysInWeekNights                  0
Adults                             0
Children                           0
Babies                             0
Meal                               0
Country                          464
MarketSegment                      0
DistributionChannel                0
IsRepeatedGuest                    0
PreviousCancellations              0
PreviousBookingsNotCanceled        0
ReservedRoomType                   0
AssignedRoomType                   0
BookingChanges                     0
DepositType                        0
Agent                           8209
Company                        36952
DaysInWaitingList                  0
CustomerType                       0
ADR                                0
R

Most of the elements in feature Company is null, so it is better to delete this feature.

In [37]:
# Drop Company from both hotel_1 & hotel_2 datasets
hotel_1 = hotel_1.drop(['Company'],axis=1)
hotel_2 = hotel_2.drop(['Company'],axis=1)

# Fill NA values using Most frequently occuring value in that column
hotel_1['Country'] = hotel_1['Country'].fillna(hotel_1['Country'].mode()[0])
hotel_1['Agent'] = hotel_1['Agent'].fillna(hotel_1['Agent'].mode()[0])

hotel_2['Country'] = hotel_2['Country'].fillna(hotel_2['Country'].mode()[0])
hotel_2['Agent'] = hotel_2['Agent'].fillna(hotel_2['Agent'].mode()[0])
hotel_2['Children'] = hotel_2['Children'].fillna(hotel_2['Children'].mode()[0])

In [38]:
print(hotel_1.isna().sum())
print(hotel_2.isna().sum())

IsCanceled                     0
LeadTime                       0
ArrivalDateYear                0
ArrivalDateMonth               0
ArrivalDateWeekNumber          0
ArrivalDateDayOfMonth          0
StaysInWeekendNights           0
StaysInWeekNights              0
Adults                         0
Children                       0
Babies                         0
Meal                           0
Country                        0
MarketSegment                  0
DistributionChannel            0
IsRepeatedGuest                0
PreviousCancellations          0
PreviousBookingsNotCanceled    0
ReservedRoomType               0
AssignedRoomType               0
BookingChanges                 0
DepositType                    0
Agent                          0
DaysInWaitingList              0
CustomerType                   0
ADR                            0
RequiredCarParkingSpaces       0
TotalOfSpecialRequests         0
ReservationStatus              0
dtype: int64
IsCanceled                    

## Data Exploration

IsCanceled: 

LeadTime:

ArrivalDateYear:

ArrivalDateMonth:

ArrivalDateWeekNumber:

ArrivalDateDayOfMonth:

StaysInWeekendNights:

StaysInWeekNights:

Adults:

Children:

Babies: 

Meal: 

Country: 

MarketSegment:

DistributionChannel:

IsRepeatedGuest:

PreviousCancellations:

PreviousBookingsNotCanceled: 

ReservedRoomType:

AssignedRoomType:

BookingChanges:

DepositType: 

Agent: 

DaysInWaitingList:


CustomerType:

ADR:

RequiredCarParkingSpaces:

TotalOfSpecialRequests:

ReservationStatus:

### Feature Engineering

The traveling time is represented by both year, date, month, and week. I think only week and year is sufficient.



In [ ]:
# drop arrival date month
hotel_1 = hotel_1.drop(['ArrivalDateMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateMonth'],axis=1)

# drop arrival date day of month
hotel_1 = hotel_1.drop(['ArrivalDateDayOfMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateDayOfMonth'],axis=1)

In [47]:
for col in hotel_1.columns:
    print(col + " : " + str(len(hotel_1[col].unique())) + " features ")
    print("Examples: " + str(hotel_1[col].unique() if len(hotel_1[col].unique()) < 5 else hotel_1[col].unique()[:5]) + "\n")

IsCanceled : 2 features 
Examples: [0 1]

LeadTime : 412 features 
Examples: [342 737   7  13  14]

ArrivalDateYear : 3 features 
Examples: [2015 2016 2017]

ArrivalDateMonth : 12 features 
Examples: ['July' 'August' 'September' 'October' 'November']

ArrivalDateWeekNumber : 53 features 
Examples: [27 28 29 30 31]

ArrivalDateDayOfMonth : 31 features 
Examples: [1 2 3 4 5]

StaysInWeekendNights : 16 features 
Examples: [0 1 2 4 3]

StaysInWeekNights : 31 features 
Examples: [0 1 2 3 4]

Adults : 14 features 
Examples: [ 2  1  3  4 40]

Children : 5 features 
Examples: [ 0  1  2 10  3]

Babies : 3 features 
Examples: [0 1 2]

Meal : 5 features 
Examples: ['BB       ' 'FB       ' 'HB       ' 'SC       ' 'Undefined']

Country : 125 features 
Examples: ['PRT' 'GBR' 'USA' 'ESP' 'IRL']

MarketSegment : 6 features 
Examples: ['Direct' 'Corporate' 'Online TA' 'Offline TA/TO' 'Complementary']

DistributionChannel : 4 features 
Examples: ['Direct' 'Corporate' 'TA/TO' 'Undefined']

IsRepeatedGues